In [1]:
import pandas as pd
import xarray as xr
import openpyxl
import numpy as np

In [66]:
activities = [
    "Paddy rice",
    "Wheat",
    "Cereal grains nec",
    "Vegetables, fruit, nuts",
    "Oil seeds",
    "Sugar cane, sugar beet",
    "Plant-based fibers",
    "Crops nec",
    "Bovine cattle, sheep and goats, horses",
    "Animal products nec",
    "Raw milk",
    "Wool, silk-worm cocoons",
    "Forestry",
    "Fishing",
    "Coal",
    "Oil",
    "Gas",
    "Other Extraction (formerly omn Minerals nec)",
    "Bovine meat products",
    "Meat products nec",
    "Vegetable oils and fats",
    "Dairy products",
    "Processed rice",
    "Sugar",
    "Food products nec",
    "Beverages and tobacco products",
    "Textiles",
    "Wearing apparel",
    "Leather products",
    "Wood products",
    "Paper products, publishing",
    "Petroleum, coal products",
    "Chemical products",
    "Basic pharmaceutical products",
    "Rubber and plastic products",
    "Mineral products nec",
    "Ferrous metals",
    "Metals nec",
    "Metal products",
    "Computer, electronic and optical products",
    "Electrical equipment",
    "Machinery and equipment nec",
    "Motor vehicles and parts",
    "Transport equipment nec",
    "Manufactures nec",
    "Electricity transmission and distribution",
    "Gas manufacture, distribution",
    "Water",
    "Construction",
    "Trade",
    "Accommodation, Food and service activities",
    "Transport nec",
    "Water transport",
    "Air transport",
    "Warehousing and support activities",
    "Communication",
    "Financial services nec",
    "Insurance (formerly isr)",
    "Real estate activities",
    "Business services nec",
    "Recreational and other services",
    "Public Administration and defense",
    "Education",
    "Human health and social work activities",
    "Dwellings",
    "Nuclear power",
    "Coal power baseload",
    "Gas power baseload",
    "Wind power",
    "Hydro power baseload",
    "Oil power baseload",
    "Other baseload",
    "Gas power peakload",
    "Hydro power peakload",
    "Oil power peakload",
    "Solar power",
]

In [2]:
path = "data/GTAP_raw_data.xlsx"
# book = openpyxl.open(path)

sheet_names = [
    "Country Codes",
    "Commodities",
    "Labor Types",
    "Factor Types",
    "Primary Factor Purchases, BP",
    "Primary Factor Purchases, PP",
    "Endowment Supply Value (Net)",
    "Factor Subsidy Payments (Gross)",
    "Factor Employment Tax Rev",
    "Make Matrix, After Tax",
    "Make Matrix, Before Tax",
    "Population",
    "Net output tax",
    "Net Saving",
    "Imports, CIF Price",
    "Capital Depreciation",
    "Domestic Purchases by Firms, BP",
    "Domestic Purchases by Firms, PP",
    "Domestic Purchases by Gov, BP",
    "Domestic Purchases by Gov, PP",
    "Domestic Purchases by Inv, BP",
    "Domestic Purchases by Inv, PP",
    "Domestic Purchases by HH, BP",
    "Domestic Purchases by HH, PP",
    "Capital Stock",
    "Import Purchases, by Firms, BP",
    "Import Purchases, by Firms, PP",
    "Import Purchases, by Gov, BP",
    "Import Purchases, by Gov, PP",
    "Import Purchases, by Inv, BP",
    "Import Purchases, by Inv, PP",
    "Import Purchases, by HH, BP",
    "Import Purchases, by HH, PP",
]

In [3]:
country_codes = pd.read_excel(
    path, sheet_name="Country Codes", index_col=0, header=None, names=["code", "country"]
)
commodity_codes = pd.read_excel(
    path, sheet_name="Commodities", index_col=0, header=None, names=["code", "commodity"]
)
labor_codes = pd.read_excel(
    path, sheet_name="Labor Types", index_col=0, header=None, names=["code", "labor"]
)
factor_codes = pd.read_excel(
    path, sheet_name="Factor Types", index_col=0, header=None, names=["code", "factor"]
)
agent_codes = {"Firms": "firms", "Gov": "government", "HH": "household", "Inv": "investment"}
price_codes = {"BP": "base price", "PP": "purchaser price"}

names = ["country", "commodity", "labor", "factor", "agent", "price"]
codes = [country_codes, commodity_codes, labor_codes, factor_codes, agent_codes, price_codes]


CODES = {
    name: x.iloc[:, 0].to_dict() if isinstance(x, pd.DataFrame) else x
    for name, x in zip(names, codes)
}

In [149]:
commodity_to_code = {v: k for k, v in CODES["commodity"].items()}
activity_codes = [commodity_to_code[x] for x in activities]

In [199]:
coords_by_sheet = {
    "Primary Factor Purchases, BP": {
        "index": ("factor", "commodity"),
        "columns": ("country",),
        "groups": ("price",),
    },
    "Primary Factor Purchases, PP": {
        "index": ("factor", "commodity"),
        "columns": ("country",),
        "groups": ("price",),
    },
    "Endowment Supply Value (Net)": {
        "index": ("factor", "commodity"),
        "columns": ("country",),
        "groups": None,
    },
    "Factor Subsidy Payments (Gross)": {
        "index": ("factor", "commodity"),
        "columns": ("country",),
        "groups": None,
    },
    "Factor Employment Tax Rev": {
        "index": ("factor", "commodity"),
        "columns": ("country",),
        "groups": None,
    },
    "Make Matrix, After Tax": {
        "index": ("commodity", "commodity_bis"),
        "columns": ("country",),
        "groups": ("tax",),
    },
    "Make Matrix, Before Tax": {
        "index": ("commodity", "commodity_bis"),
        "columns": ("country",),
        "groups": ("tax",),
    },
    "Population": {"index": ("country",), "columns": None, "groups": None},
    "Net output tax": {
        "index": ("commodity", "commodity_bis"),
        "columns": ("country",),
        "groups": None,
    },
    "Net Saving": {"index": ("country",), "columns": None, "groups": None},
    "Imports, CIF Price": {
        "index": ("commodity", "country_bis"),
        "columns": ("country",),
        "groups": None,
    },
    "Capital Depreciation": {"index": ("country",), "columns": None, "groups": None},
    "Domestic Purchases by Firms, BP": {
        "index": ("commodity", "commodity_bis"),
        "columns": ("country",),
        "groups": ("agent", "price"),
    },
    "Domestic Purchases by Firms, PP": {
        "index": ("commodity", "commodity_bis"),
        "columns": ("country",),
        "groups": ("agent", "price"),
    },
    "Domestic Purchases by Gov, BP": {
        "index": ("commodity",),
        "columns": ("country",),
        "groups": ("agent", "price"),
    },
    "Domestic Purchases by Gov, PP": {
        "index": ("commodity",),
        "columns": ("country",),
        "groups": ("agent", "price"),
    },
    "Domestic Purchases by Inv, BP": {
        "index": ("commodity",),
        "columns": ("country",),
        "groups": ("agent", "price"),
    },
    "Domestic Purchases by Inv, PP": {
        "index": ("commodity",),
        "columns": ("country",),
        "groups": ("agent", "price"),
    },
    "Domestic Purchases by HH, BP": {
        "index": ("commodity",),
        "columns": ("country",),
        "groups": ("agent", "price"),
    },
    "Domestic Purchases by HH, PP": {
        "index": ("commodity",),
        "columns": ("country",),
        "groups": ("agent", "price"),
    },
    "Capital Stock": {"index": ("country",), "columns": None, "groups": None},
    "Import Purchases, by Firms, BP": {
        "index": ("commodity", "commodity_bis"),
        "columns": ("country",),
        "groups": ("agent", "price"),
    },
    "Import Purchases, by Firms, PP": {
        "index": ("commodity", "commodity_bis"),
        "columns": ("country",),
        "groups": ("agent", "price"),
    },
    "Import Purchases, by Gov, BP": {
        "index": ("commodity",),
        "columns": ("country",),
        "groups": ("agent", "price"),
    },
    "Import Purchases, by Gov, PP": {
        "index": ("commodity",),
        "columns": ("country",),
        "groups": ("agent", "price"),
    },
    "Import Purchases, by Inv, BP": {
        "index": ("commodity",),
        "columns": ("country",),
        "groups": ("agent", "price"),
    },
    "Import Purchases, by Inv, PP": {
        "index": ("commodity",),
        "columns": ("country",),
        "groups": ("agent", "price"),
    },
    "Import Purchases, by HH, BP": {
        "index": ("commodity",),
        "columns": ("country",),
        "groups": ("agent", "price"),
    },
    "Import Purchases, by HH, PP": {
        "index": ("commodity",),
        "columns": ("country",),
        "groups": ("agent", "price"),
    },
}

sheet_groups = [
    ("Primary Factor Purchases, BP", "Primary Factor Purchases, PP"),
    ("Endowment Supply Value (Net)",),
    ("Factor Subsidy Payments (Gross)",),
    ("Factor Employment Tax Rev",),
    ("Make Matrix, After Tax", "Make Matrix, Before Tax"),
    ("Population",),
    ("Net output tax",),
    ("Net Saving",),
    ("Imports, CIF Price",),
    ("Capital Depreciation",),
    (
        "Domestic Purchases by Firms, BP",
        "Domestic Purchases by Firms, PP",
        "Domestic Purchases by Gov, BP",
        "Domestic Purchases by Gov, PP",
        "Domestic Purchases by Inv, BP",
        "Domestic Purchases by Inv, PP",
        "Domestic Purchases by HH, BP",
        "Domestic Purchases by HH, PP",
    ),
    ("Capital Stock",),
    (
        "Import Purchases, by Firms, BP",
        "Import Purchases, by Firms, PP",
        "Import Purchases, by Gov, BP",
        "Import Purchases, by Gov, PP",
        "Import Purchases, by Inv, BP",
        "Import Purchases, by Inv, PP",
        "Import Purchases, by HH, BP",
        "Import Purchases, by HH, PP",
    ),
]

In [200]:
def repeat(f, x, n):
    for _ in range(n):
        x = f(x)
    return x


data_vars = {}
for sheets in sheet_groups:
    group_stack = []
    group_names = []
    group_dims = []
    for sheet in sheets:
        tokens = None
        coords = coords_by_sheet[sheet]
        n_headers = None if coords["columns"] is None else len(coords["columns"])
        n_index = len(coords["index"])

        df = pd.read_excel(path, sheet_name=sheet, index_col=list(range(n_index)))
        if n_headers is None:
            df.columns = [sheet]

        if n_index > 1:
            df.index.names = coords["index"]
        else:
            df.index.name = coords["index"][0]

        if n_headers is None:
            name = sheet
        else:
            if n_headers > 1:
                df.columns.names = coords["columns"]
            else:
                df.columns.name = coords["columns"][0]

            tokens = [
                x.strip()
                for token in sheet.split(",")
                for x in token.split("by")
                if len(x.strip()) > 0
            ]
            groups = coords["groups"]
            index_cols = df.index.names
            df = df.reset_index(drop=False)
            if groups is None:
                name = " ".join(tokens)
            else:
                name = tokens.pop(0)
                for group, token in zip(groups, tokens):
                    df[group] = token
                    index_cols += [group]
            if name.title() in ["Domestic Purchases", "Import Purchases"]:
                if "commodity_bis" not in df.columns:
                    df["commodity_bis"] = df.commodity
                    index_cols += ["commodity_bis"]
            df.set_index(sorted(index_cols), inplace=True)
            df = repeat(lambda x: x.stack(), df, n_headers if n_headers is not None else 1)

        group_stack.append(df)
        group_dims.append(tokens)
    if len(group_stack) > 1:
        data = pd.concat(group_stack).to_xarray()
    else:
        temp = group_stack[0]
        if isinstance(temp, pd.DataFrame):
            temp = temp.iloc[:, 0]
        data = temp.to_xarray()
    data_vars[name.title()] = data

In [201]:
gtap = xr.Dataset(data_vars)

In [202]:
gtap

<xarray.Dataset>
Dimensions:                          (commodity: 76, factor: 8, price: 2,
                                      country: 160, commodity_bis: 76, tax: 2,
                                      country_bis: 160, agent: 4)
Coordinates:
  * commodity                        (commodity) object 'AFS' 'ATP' ... 'WindBL'
  * factor                           (factor) object 'Capital' ... 'tech_aspros'
  * price                            (price) object 'BP' 'PP'
  * country                          (country) object 'AFG' 'ALB' ... 'ZWE'
  * commodity_bis                    (commodity_bis) object 'AFS' ... 'WindBL'
  * tax                              (tax) object 'After Tax' 'Before Tax'
  * country_bis                      (country_bis) object 'AFG' 'ALB' ... 'ZWE'
  * agent                            (agent) object 'Firms' 'Gov' 'HH' 'Inv'
Data variables: (12/13)
    Primary Factor Purchases         (commodity, factor, price, country) float64 ...
    Endowment Supply Value (Net)     (commodity, factor, country) float64 138...
    Factor Subsidy Payments (Gross)  (commodity, factor, country) float64 nan...
    Factor Employment Tax Rev        (commodity, factor, country) float64 0.0...
    Make Matrix                      (commodity, commodity_bis, tax, country) float64 ...
    Population                       (country) float64 36.3 2.873 ... 14.24
    ...                               ...
    Net Saving                       (country) float64 -9.284e+03 ... -3.556e+03
    Imports Cif Price                (commodity, country_bis, country) float64 ...
    Capital Depreciation             (country) float64 2.871e+03 ... 3.126e+03
    Domestic Purchases               (agent, commodity, commodity_bis, price, country) float64 ...
    Capital Stock                    (country) float64 7.177e+04 ... 7.815e+04
    Import Purchases                 (agent, commodity, commodity_bis, price, country) float64 ...

In [253]:
sam_path = "data/albania_sam.csv"
df = pd.read_csv(sam_path, header=[0, 1], index_col=[0, 1]).fillna(0)

In [254]:
labor = (
    gtap.sel(country="ALB", price="BP")["Primary Factor Purchases"]
    .sel(factor=labor_codes.index.values.tolist())
    .sum(dim="factor")
)
capital = gtap.sel(country="ALB", price="BP")["Primary Factor Purchases"].sel(factor="Capital")
resource = gtap.sel(country="ALB", price="BP")["Primary Factor Purchases"].sel(factor="NatlRes")
land = gtap.sel(country="ALB", price="BP")["Primary Factor Purchases"].sel(factor="Land")
X = gtap.sel(country="ALB", price="BP", agent="Firms")["Domestic Purchases"]
C_bp = gtap.sel(country="ALB", price="BP", agent="HH")["Domestic Purchases"]
C_pp = gtap.sel(country="ALB", price="PP", agent="HH")["Domestic Purchases"]

I_bp = gtap.sel(country="ALB", price="BP", agent="Inv")["Domestic Purchases"]
I_pp = gtap.sel(country="ALB", price="PP", agent="Inv")["Domestic Purchases"]

C_G = gtap.sel(country="ALB", price="BP", agent="Gov")["Domestic Purchases"]
S = gtap["Net Saving"].sel(country="ALB").values

K_tax_rev = gtap["Factor Employment Tax Rev"].sel(country="ALB", factor="Capital")
L_tax_rev = (
    gtap["Factor Employment Tax Rev"]
    .sel(country="ALB", factor=labor_codes.index.values.tolist())
    .sum(dim="factor")
)

In [257]:
df.loc[("Factor", "Labor"), "Activities"] = labor.sel(commodity=activity_codes).values
df.loc[("Factor", "Capital"), "Activities"] = capital.sel(commodity=activity_codes).values
df.loc["Production", "Activities"] = X.sel(
    commodity=activity_codes, commodity_bis=activity_codes
).values
df.loc["Production", ("Institution", "Household")] = np.diag(
    C_bp.sel(commodity=activity_codes, commodity_bis=activity_codes).values
)
df.loc["Production", ("Institution", "Govt")] = np.diag(
    C_G.sel(commodity=activity_codes, commodity_bis=activity_codes).values
)
df.loc[("Other", "Capital Accumulation"), ("Institution", "Household")] = S
df.loc[("Use Tax", "Capital"), "Production"] = K_tax_rev.sel(commodity=activity_codes).values
df.loc[("Use Tax", "Labor"), "Production"] = L_tax_rev.sel(commodity=activity_codes).values
df.loc["Sales Tax", ("Institution", "Household")] = np.diag(
    (C_pp - C_bp).sel(commodity=activity_codes, commodity_bis=activity_codes)
)
df.loc["Production", ("Other", "Capital Accumulation")] = np.diag(I_pp)

In [259]:
df.to_csv(sam_path)